<a href="https://colab.research.google.com/github/zhulkhoirr/Steam-Recommendation-System/blob/main/SteamRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistem Rekomendasi Game Pada Platform Steam

## 1. Download Datasets Dari Kaggle

In [ ]:
from google.colab import files
files.upload()

In [3]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d antonkozyriev/game-recommendations-on-steam

Dataset URL: https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam
License(s): CC0-1.0
 98% 648M/660M [00:08<00:00, 144MB/s]
100% 660M/660M [00:08<00:00, 79.2MB/s]


In [7]:
!unzip -q game-recommendations-on-steam.zip

## 2. Membaca dan Memahami Data

In [8]:
import numpy as np
import pandas as pd

In [9]:
games_df = pd.read_csv('games.csv')

games_df.sample(5)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
23144,19200,Tank Universal,2008-08-21,True,False,False,Mixed,60,328,4.99,4.99,0.0,True
46244,1756650,Lockes The Thief,2023-02-08,True,False,False,Positive,90,10,7.99,7.99,0.0,True
11763,208288,Train Simulator: Munich-Augsburg Route Add-On,2012-10-10,True,False,False,Mixed,65,69,19.99,19.99,0.0,True
22072,1363540,DIRT 5 - Gameplay Booster Pack,2020-11-05,True,False,False,Mostly Negative,33,12,5.99,5.99,0.0,True
16404,502230,Monsters' Den: Book of Dread,2016-07-19,True,False,False,Very Positive,92,163,4.99,4.99,0.0,True


In [10]:
len(games_df)

50872

In [11]:
sort_games_df = games_df.sort_values(by='user_reviews', ascending=False)

sort_games_df.head(10)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
14398,730,Counter-Strike: Global Offensive,2012-08-21,True,True,True,Very Positive,88,7494460,15.0,0.0,0.0,True
47770,578080,PUBG: BATTLEGROUNDS,2017-12-21,True,False,False,Mixed,57,2217226,0.0,0.0,0.0,True
13176,570,Dota 2,2013-07-09,True,True,True,Very Positive,82,2045628,0.0,0.0,0.0,True
12717,271590,Grand Theft Auto V,2015-04-13,True,False,False,Very Positive,86,1484122,0.0,0.0,0.0,True
14535,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,True,False,False,Very Positive,86,993312,20.0,0.0,0.0,True
47380,440,Team Fortress 2,2007-10-10,True,True,True,Very Positive,93,985819,0.0,0.0,0.0,True
13035,105600,Terraria,2011-05-16,True,True,True,Overwhelmingly Positive,97,943413,10.0,0.0,0.0,True
15363,4000,Garry's Mod,2006-11-29,True,True,True,Overwhelmingly Positive,96,853733,10.0,0.0,0.0,True
13173,252490,Rust,2018-02-08,True,True,False,Very Positive,87,786668,40.0,0.0,0.0,True
14376,1172470,Apex Legends™,2020-11-04,True,False,False,Very Positive,80,713182,0.0,0.0,0.0,True


In [12]:
users_df = pd.read_csv('users.csv')

users_df.sample(5)

,user_id,products,reviews
4514893,11211769,139,1
12155567,3126277,19,1
14128912,8848517,103,1
8707643,13208974,27,2
8409750,9011097,48,0


In [13]:
recommendations_df = pd.read_csv('recommendations.csv')

recommendations_df.sample(5)

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
18862866,346110,0,0,2017-01-29,True,481.1,8382096,18862866
12544942,292030,0,0,2022-11-20,True,74.6,10482087,12544942
12476094,1089980,0,0,2022-07-09,True,10.2,3724849,12476094
4140762,252490,0,0,2014-01-23,True,708.6,11857798,4140762
26146259,219740,0,0,2020-02-03,True,119.4,12689554,26146259


In [14]:
import json

with open('games_metadata.json', 'r') as file:
    data = file.read()

json_objects = data.split('\n')

data_list = []
for obj in json_objects:
    if obj.strip():
        data_list.append(json.loads(obj))

games_metadata_df = pd.DataFrame(data_list)
games_metadata_df.sample(5)


,app_id,description,tags
26030,544820,Classic old-school hardcore two dimensional ho...,"[Adventure, Indie, Platformer]"
35370,396530,Adventure horror with many puzzles and an inte...,"[Adventure, Indie, Horror, Puzzle]"
17858,390310,"Once upon a time, a little boy from our world ...","[Strategy, RPG, Simulation, Cute, Great Soundt..."
15280,376210,,[]
27484,1669330,《残世界的鸢尾花》是一款近未来都市废土风的RPG游戏，讲述了三个素不相识的残世界幸存者结伴逃...,"[Indie, Story Rich, Cinematic, RPG, Emotional,..."


## 3. Mempersiapkan Data

In [15]:
recommendations_df = pd.read_csv('recommendations.csv', usecols=['app_id', 'is_recommended', 'user_id'])

recommendations_df.head(5)

,app_id,is_recommended,user_id
0,975370,True,51580
1,304390,False,2586
2,1085660,True,253880
3,703080,True,259432
4,526870,True,23869


In [16]:
n_recommendations = recommendations_df['user_id'].value_counts()
print(n_recommendations)

user_id
11764552    6045
5112758     4152
11656130    3840
5669734     3479
11553593    3392
            ... 
5549796        1
160135         1
10673403       1
4073692        1
2306695        1
Name: count, Length: 13781059, dtype: int64


In [17]:
n_recommendations = recommendations_df['user_id'].value_counts()
recommendations_df = recommendations_df[recommendations_df['user_id'].isin(n_recommendations[n_recommendations >=50].index)].copy()
recommendations_df.head(5)

,app_id,is_recommended,user_id
58,602960,True,737481
138,570940,False,5136277
151,1196590,True,5935019
203,304390,False,10567745
209,244850,True,11143713


In [18]:
print("Jumlah duplikasi:",  recommendations_df.duplicated().sum())

Jumlah duplikasi: 1


In [19]:
recommendations_df.drop_duplicates(inplace=True)
print("Jumlah duplikasi:",  recommendations_df.duplicated().sum())

Jumlah duplikasi: 0


In [20]:
recommendations_df.isnull().sum()

app_id            0
is_recommended    0
user_id           0
dtype: int64

In [24]:
recommendations_df['is_recommended'] = recommendations_df['is_recommended'].astype(int)

## 4. Preprocessing Data

In [25]:
user_ids = recommendations_df['user_id'].unique().tolist()
user2user_encoded = {x:i for i, x in enumerate(user_ids)}
user_encoded2user = {i:x for i, x in enumerate(user_ids)}
recommendations_df['user'] = recommendations_df['user_id'].map(user2user_encoded)
n_users = len(user2user_encoded)

app_ids = recommendations_df['app_id'].unique().tolist()
app2app_encoded = {x:i for i, x in enumerate(app_ids)}
app_encoded2app = {i:x for i, x in enumerate(app_ids)}


,app_id,is_recommended,user_id,game
58,602960,1,737481,0
138,570940,0,5136277,1
151,1196590,1,5935019,2
203,304390,0,10567745,3
209,244850,1,11143713,4
...,...,...,...,...
41154764,1818450,1,13261742,25181
41154770,700600,1,6382076,35496
41154771,362960,1,7858713,21724
41154784,633230,0,8107668,18375
